In [1]:
from bs4 import BeautifulSoup
import json
import pandas as pd
import requests
import re
import time

In [3]:
headers = {
    "USER_AGENT": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36",
    "referer": "https://fincaraiz.com.co/"
}

json_data = {
    'filter': {
        'offer': {
            'slug': [
                'sell',
            ],
        },
        'is_new': 'False',
        'locations': {
            'cities': {
                'name': ['Medellín']
            }
        },
    },
    'fields': {
        'exclude': [],
        'include': [
            'area',
            'baths.id',
            'baths.name',
            'client.client_type',
            'client.company_name',
            'client.first_name',
            'client.last_name',
            'garages.name',
            'is_new',
            'locations.cities.name',
            'locations.cities.slug',
            'locations.countries.name',
            'locations.countries.slug',
            'locations.groups.name',
            'locations.groups.slug',
            'locations.groups.subgroups.name',
            'locations.groups.subgroups.slug',
            'locations.location_point',
            'locations.neighbourhoods.name',
            'locations.neighbourhoods.slug',
            'locations.states.name',
            'locations.states.slug',
            'min_area',
            'min_price',
            'price',
            'products.configuration.tag_name',
            'products.label',
            'products.name',
            'products.slug',
            'property_id',
            'fr_property_id',
            'rooms.name',
            'title',
            'property_type.name',
            'offer.name',
            'fr_parent_property_id',
            'stratum.name',
        ],
        'limit': 25,
        'offset': 0,
        'ordering': [],
        'platform': 40,
        'with_algorithm': False,
    }
}

In [4]:
def obtener_datos(offset):
    json_data['fields']['offset'] = offset
    response = requests.post('https://api.fincaraiz.com.co/document/api/1.0/listing/search', headers=headers, json=json_data, timeout=10)
    data = response.json()
    return data.get('hits', {}).get('hits', [])

num_paginas = 400 #es la máxima cantidad de páginas que podemos extraer

properties_data = []

offset = 0

for pagina in range(num_paginas):
    hits = obtener_datos(offset)

    if not hits:
        break

    for hit in hits:
        source = hit['_source']['listing']
        property_info = {
            'area': source['area'],
            'rooms': source['rooms']['name'],
            'is_new': source['is_new'],
            'property_id': source['property_id'],
            'garages': source['garages']['name'],
            'stratum': source['stratum']['name'],
            'property_type': source['property_type'][0]['name'],
            'baths': source['baths']['name'],
            'min_price': source['min_price'],
            'price': source['price'],
            'company_name': source['client']['company_name'],
            'client_type': source['client']['client_type'],
            'neighbourhood': source['locations']['neighbourhoods'][0]['name'],
            'city': source['locations']['cities'][0]['name'],
            'latitude': source['locations']['location_point'].split()[1][1:],
            'longitude': source['locations']['location_point'].split('(')[1].split()[1].rstrip(')'),
            'zone': source['locations'].get('groups', [{'subgroups': {'name': 'No Zone'}}])[0]['subgroups']['name']
        }
        properties_data.append(property_info)

    offset += 25

df = pd.DataFrame(properties_data)
df.drop_duplicates(subset='property_id', keep='first', inplace=True)

In [7]:
df.to_csv('data.csv',index = False)